In [1]:
pip install numpy pandas matplotlib seaborn bokeh selenium

In [ ]:
import bokeh
import pandas as pd

from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.io import output_notebook
import pandas as pd
from bokeh.models import Legend
from bokeh.palettes import Spectral, brewer, Viridis256, Spectral11, Viridis
from bokeh.palettes import brewer
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.transform import linear_cmap
from bokeh.resources import INLINE
import bokeh.io
from bokeh import *
bokeh.io.output_notebook(INLINE)

from bokeh.io import output_notebook, export_svgs, save, reset_output
from bokeh import palettes
from bokeh.models import Legend
from bokeh.palettes import Spectral
from bokeh.palettes import brewer

In [ ]:
df_main = pd.read_excel("Typology of Quantum Algorithms.xlsx", sheet_name="CleanTypology")
df_main.head()

,Algorithm,Primitive used,Fundamental Math Problem,Math1,Math2,Math3,Application,Unnamed: 7,Proven / Heuristic,NISQ / LSQ,Oracular / Non-Oracular,Promise,Sampling,Computational Model,Google Scholar citations,Links to paper,Dates
0,QFT,NaN,Hidden Subgroup problem. Dynamic system: fouri...,HSP,DYS,NaN,NaN,NaN,Proven,LSQ,Non-Oracular,No,No,Circuit Model,468,[quant-ph/9511026] Quantum measurements and th...,
1,QPE,QFT,Linear Algebra problem: estimate the phase (or...,LA,DYS,NaN,First-principle computational physics/chemistry,"FCP,FCC",Proven,LSQ,Non-Oracular,No,No,Circuit Model,1150,[quant-ph/9511026] Quantum measurements and th...,20/11/1995
2,Shor,QFT,"Hidden Subgroup problem (Factorization, Discre...",HSP,NaN,NaN,"Crytanalysis-Asymmetric (Breaking RSA, Diffie-...",CR,Proven,LSQ,Non-Oracular,No,No,Circuit Model,13640,https://arxiv.org/abs/quant-ph/9508027,30/08/1995
3,HHL,QFT,Linear Algebra problem: Solving systems for li...,LA,NaN,NaN,"Machine Learning, Data science, First-principl...","ML,DS,FCP,OP,CCP",Proven,LSQ,Non-Oracular,Yes,No,Circuit Model,2554,[0811.3171] Quantum algorithm for solving line...,19/11/2018
4,VQE,NaN,Linear Algebra problem: Estimate phase of a un...,LA,DYS,NaN,First-principle computational physics/chemistr...,"FCP,FCC",Heuristic,NISQ,Non-Oracular,Νο,Yes,Circuit Model,3181,https://arxiv.org/abs/1304.3061,10/04/2013


In [ ]:
df_main.dtypes

Algorithm                   object
Primitive used              object
Fundamental Math Problem    object
Math1                       object
Math2                       object
Math3                       object
Application                 object
Unnamed: 7                  object
Proven / Heuristic          object
NISQ / LSQ                  object
Oracular / Non-Oracular     object
Promise                     object
Sampling                    object
Computational Model         object
Google Scholar citations    object
Links to paper              object
Dates                       object
dtype: object

In [ ]:
algorithm_publication_dates = df_main[["Algorithm", "Dates"]].assign(length=df_main['Dates']\
                                                            .apply(lambda x: len(str(x))))\
                                                            .dropna(subset=["Dates"])
algorithm_publication_dates = algorithm_publication_dates.query("length > 1").drop(columns=["length"])
algorithm_publication_dates.loc[:, "Dates"] = algorithm_publication_dates["Dates"].apply(str).apply(lambda x: x if x.count("/") == 2 else x[:5] + "/" + x[-4:])
algorithm_publication_dates = algorithm_publication_dates[algorithm_publication_dates['Dates'].apply(lambda x: str(x).count("/")) == 2]
algorithm_publication_dates

,Algorithm,Dates
1,QPE,20/11/1995
2,Shor,30/08/1995
3,HHL,19/11/2018
4,VQE,10/04/2013
5,Amplitude amplification,15/05/2000
...,...,...
140,Quantum Full configuration interaction (FCI),30/09/2021
141,Quantum Density funtional theory(DFT) method,04/04/2022
142,Circuit formulation of adiabatic algorithm for...,06/04/2022
143,variational quantum algorithm for protein fold...,06/08/2019


In [ ]:
def correct_date(x):
    splitted = x.split("-")
    if int(splitted[1]) > 12:
        return splitted[1] + "-" + splitted[0] + "-" + splitted[-1]

    return "-".join(splitted)

algorithm_publication_dates.loc[:, "Year"] = pd.to_datetime(algorithm_publication_dates['Dates'].apply(lambda x: x.replace("/", "-")).apply(correct_date),
                                                             format="%d-%m-%Y", dayfirst=True)
algorithm_publication_dates.sort_values(by=["Year"], ascending=True)
algorithm_publication_dates

,Algorithm,Dates,Year
1,QPE,20/11/1995,1995-11-20
2,Shor,30/08/1995,1995-08-30
3,HHL,19/11/2018,2018-11-19
4,VQE,10/04/2013,2013-04-10
5,Amplitude amplification,15/05/2000,2000-05-15
...,...,...,...
140,Quantum Full configuration interaction (FCI),30/09/2021,2021-09-30
141,Quantum Density funtional theory(DFT) method,04/04/2022,2022-04-04
142,Circuit formulation of adiabatic algorithm for...,06/04/2022,2022-04-06
143,variational quantum algorithm for protein fold...,06/08/2019,2019-08-06


In [ ]:
publication_timeline = algorithm_publication_dates.drop(columns=["Dates"]).set_index("Year").resample("1Y").count().reset_index()
publication_timeline.head()

,Year,Algorithm
0,1995-12-31,2
1,1996-12-31,2
2,1997-12-31,0
3,1998-12-31,1
4,1999-12-31,1


In [2]:
mapping = {
            "HSP": "Hidden Subgroup Problem",
            "LA": "Linear Algebra",
            "COM": "Combinatorial",
            "OPT": "Optimization",
            "DYS": "Dynamical System",
            "ST": "Statistics",
            "SP": "Stochastic process",
          }

In [ ]:
df = pd.read_excel("Typology of Quantum Algorithms.xlsx", sheet_name="CleanTypology")\
            .dropna(subset=['Algorithm'])\
            .iloc[:, :7]\
            .drop(columns=['Primitive used', 'Fundamental Math Problem', 'Application'])
df.head()

,Algorithm,Math1,Math2,Math3
0,QFT,HSP,DYS,NaN
1,QPE,LA,DYS,NaN
2,Shor,HSP,NaN,NaN
3,HHL,LA,NaN,NaN
4,VQE,LA,DYS,NaN


In [3]:
df.loc[:, "categories"] = df.iloc[:,1:].apply(lambda row: {row[l] for l in row.index if isinstance(row[l], str)}, axis=1)
math_count = df[["Algorithm", "categories"]].explode("categories")\
                                                    .dropna(subset=['categories'])\
                                                    .groupby("categories", as_index=False)\
                                                    .count()\
                                                    .sort_values(by=['Algorithm'], ascending=True)
math_count

In [ ]:
new_df = df[["Algorithm"]].copy()

for category in mapping:
    new_df.loc[:, category] = df['categories'].apply(lambda collection: int(category in collection))
new_df.head()

,Algorithm,HSP,LA,COM,CV,DYS,ST,SP
0,QFT,1,0,0,0,1,0,0
1,QPE,0,1,0,0,1,0,0
2,Shor,1,0,0,0,0,0,0
3,HHL,0,1,0,0,0,0,0
4,VQE,0,1,0,0,1,0,0


In [ ]:
#colors = {
#    "SP": "#458B74",
#    "ST": "#76EEC6",
#    "LA": "#66CD00",
#    "COM": "#1C86EE",
#    "DYS": "#00CED1",
#    "HSP": "#FF8000",
#    "OPT": "#FF0000"
#}


In [ ]:
mathcat = ['SP', 'ST', 'LA', 'COM', 'DYS', 'HSP', 'OPT']

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.io import output_notebook
import pandas as pd
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256, Spectral11, Viridis
from bokeh.io import output_notebook, export_svgs, save
from bokeh import palettes
from bokeh.models import Legend
from bokeh.palettes import Spectral
from bokeh.palettes import brewer

In [4]:
new_df.set_index('Algorithm').sum()

In [ ]:
algorithm_publication_dates

,Algorithm,Dates,Year
1,QPE,20/11/1995,1995-11-20
2,Shor,30/08/1995,1995-08-30
3,HHL,19/11/2018,2018-11-19
4,VQE,10/04/2013,2013-04-10
5,Amplitude amplification,15/05/2000,2000-05-15
...,...,...,...
140,Quantum Full configuration interaction (FCI),30/09/2021,2021-09-30
141,Quantum Density funtional theory(DFT) method,04/04/2022,2022-04-04
142,Circuit formulation of adiabatic algorithm for...,06/04/2022,2022-04-06
143,variational quantum algorithm for protein fold...,06/08/2019,2019-08-06


In [10]:
timeseries_df = new_df.merge(algorithm_publication_dates.drop(columns=["Dates"]), on="Algorithm", how="right").set_index("Year").drop(columns=['Algorithm']).resample("1Y").sum().cumsum().reset_index()
timeseries_df.head()

In [9]:
timeseries_df.loc[:, 'total'] = timeseries_df[mathcat].sum(axis=1)
timeseries_df.head()

In [6]:
timeseries_df.loc[:, 'Year'] = timeseries_df['Year'].astype(str)
timeseries_df.head()

In [5]:
pip install --upgrade bokeh

In [7]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

colors = brewer['Spectral'][len(mathcat)]
source = ColumnDataSource(data=timeseries_df)

#color_palette = ['darkblue', 'darkgreen', 'darkred', 'orange', 'purple','yellow', 'red']

reset_output()
output_notebook()

p = figure(x_axis_type="datetime", title='Math Categories', background_fill_color="#f9f9f9")

renderers = p.varea_stack(mathcat,
                    x='Year',
                    #color=colors,
              # legend_label=domains,
              source=source)


legend = Legend(items=[(label, [renderer]) for label, renderer in zip(mathcat, renderers)],
                location='top_left', click_policy='hide')
p.add_layout(legend, 'right')

p.title.align = 'center'
p.title.text_font_size = '20pt'

p.xaxis.axis_label = 'Year'

p.yaxis.axis_label = 'Number Math. Categ.'

p.title.text_font_size = "18pt"
p.axis.major_label_text_font_size = "10pt"
p.legend.label_text_font_size = "8pt"
p.legend.spacing = 10
p.legend.padding = 20

# Customize grid lines
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "gray"
p.ygrid.grid_line_alpha = 0.5
p.ygrid.grid_line_dash = [6, 4]

p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style = "italic"
p.yaxis.axis_label_text_font_style = "italic"

output_file("stacked_math_cat.html")
# save(p, title="stacked_application_domain.png")
show(p)

In [ ]:
from bokeh.palettes import brewer

color_palette = brewer['Spectral'][len(mathcat)]
color_palette

('#3288bd', '#99d594', '#e6f598', '#ffffbf', '#fee08b', '#fc8d59', '#d53e4f')

In [ ]:
#pip install --upgrade bokeh==2.4.3
import bokeh.io
from bokeh.resources import INLINE
bokeh.io.output_notebook(INLINE)

In [8]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Legend
from bokeh.io import output_file, output_notebook

# Assuming you have your data and color_palette defined earlier
color_palette = brewer['Spectral'][len(mathcat)]

source = ColumnDataSource(data=timeseries_df)


reset_output()
output_notebook()

p = figure(plot_width=900, x_axis_type="datetime", plot_height=600, title='Application Domains', background_fill_color="#f9f9f9")



renderers = p.varea_stack(mathcat,
                    x='Year',
                    color=color_palette,
              # legend_label=domains,
              source=source)

# Customize the legend position and style
legend = Legend(
    items=[(label, [renderer]) for label, renderer in zip(mathcat, renderers)],
    location='top_left',
    click_policy='hide'
)
p.add_layout(legend, 'right')

p.title.align = 'center'
p.title.text_font_size = '20pt'

p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Number Math. Categ.'

p.title.text_font_size = "18pt"
p.axis.major_label_text_font_size = "10pt"
p.legend.label_text_font_size = "8pt"
p.legend.spacing = 10
p.legend.padding = 20

# Customize grid lines
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "gray"
p.ygrid.grid_line_alpha = 0.5
p.ygrid.grid_line_dash = [6, 4]

p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style = "italic"
p.yaxis.axis_label_text_font_style = "italic"

# Save or show the plot
output_file("stacked_math_cat.html")
show(p)
